In [ ]:
# Устанавливаем библиотеки
import numpy as np
import tensorflow as tf
import random
from google.colab import drive
import pickle

# Монтируем Google Drive, чтобы сохранить модель в нем
drive.mount('/content/drive')

# Функция для генерации случайных состояний доски
def generate_random_state():
    board = [0] * 9
    for _ in range(random.randint(0, 5)):
        empty_indices = [i for i, cell in enumerate(board) if cell == 0]
        if not empty_indices:
            break
        move = random.choice(empty_indices)
        board[move] = 1 if random.choice([True, False]) else -1
    return board

# Функция для генерации обучающих данных
def generate_training_data(num_samples=150000):
    states = []
    moves = []
    for _ in range(num_samples):
        state = generate_random_state()
        empty_indices = [i for i, cell in enumerate(state) if cell == 0]

        # Если доска заполнена, пропускаем это состояние
        if not empty_indices:
            continue

        # Генерируем случайный корректный ход
        move = [0] * 9
        best_move = random.choice(empty_indices)
        move[best_move] = 1

        states.append(state)
        moves.append(move)

    return np.array(states), np.array(moves)

# Генерация данных
print("Генерация обучающих данных...")
X, y = generate_training_data(150000)  # Увеличиваем количество данных до 150,000

# Создание более сложной нейросети
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(9,)),  # Входной слой с размерностью 9
    tf.keras.layers.Dense(128, activation='relu'),  # Увеличиваем количество нейронов
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')  # 9 выходов для каждой клетки на доске
])

# Компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
print("Обучение модели...")
history = model.fit(X, y, epochs=500, batch_size=64, verbose=1)

# Сохранение обученной модели в Google Drive
model_save_path = '/content/drive/MyDrive/tic_tac_toe_model.h5'  # Укажите путь для сохранения
model.save(model_save_path)
print(f"Модель сохранена в Google Drive как {model_save_path}")

# Сохранение обучающих данных
training_data_save_path = '/content/drive/MyDrive/training_data_for_tictac.pkl'  # Укажите путь для сохранения данных
with open(training_data_save_path, 'wb') as f:
    pickle.dump((X, y), f)
print(f"Обучающие данные сохранены в Google Drive как {training_data_save_path}")

# Теперь вы можете скачать модель и обучающие данные на свой ПК
